In [181]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import pandas as pd

In [182]:
train_data = pd.read_csv('./mnist_train.csv', sep=',', header=None)
train_labels = train_data[0]
train_data = train_data.drop(0, axis=1)

test_data = pd.read_csv('./mnist_test.csv', sep=',', header=None)
test_labels = test_data[0]
test_data = test_data.drop(0, axis=1)

train_data = torch.FloatTensor(train_data.to_numpy())
train_labels = torch.LongTensor(train_labels.to_numpy())
test_data = torch.FloatTensor(test_data.to_numpy())

In [183]:
input_size = 784
output_size = 10
hidden_size = 50

epochs = 40
batch_size = 200
learning_rate = 0.00005

In [184]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [185]:
my_net = Net()

In [186]:
optimizer = torch.optim.SGD(my_net.parameters(), lr=learning_rate, momentum=0.9)
loss = nn.CrossEntropyLoss()

In [187]:
for i in range(epochs):
    for j in range(0, train_data.shape[0], batch_size):
        data_minibatch = Variable(train_data[j : j+batch_size])
        label_minibatch = Variable(train_labels[j: j+batch_size])
        optimizer.zero_grad()
        net_out = my_net(data_minibatch)
        net_loss = loss(net_out, label_minibatch)
        net_loss.backward()
        optimizer.step()

C:\Users\ethan\AppData\Local\Temp\ipykernel_24316\966333629.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [188]:
activations = []

def get_activation():
    def hook(model, input, output):
        activations.append(output.detach().numpy())
    return hook

In [192]:
my_net.fc1.register_forward_hook(get_activation())
my_net.fc2.register_forward_hook(get_activation())
my_net.fc3.register_forward_hook(get_activation())
out = my_net(test_data)
print(activations)
print(len(activations[0]))

[array([[ -30.140766 ,  -33.977783 ,  -73.072754 , ...,    6.990468 ,
         -65.217285 ,    4.2281456],
       [-112.95897  ,  -48.209785 ,  -37.362717 , ...,  -89.13971  ,
         -81.17749  ,  -54.68694  ],
       [ -52.05144  ,  -48.533146 ,  -66.3577   , ...,    6.1681223,
         -21.432804 ,  -67.18391  ],
       ...,
       [-113.04425  , -103.617645 , -130.34914  , ...,  146.3792   ,
         -75.933075 , -206.44609  ],
       [ -86.17983  ,   -9.884845 ,  -93.59358  , ...,  224.71104  ,
         -93.19644  , -140.32481  ],
       [-160.26582  ,   -2.186168 , -124.72555  , ...,  -89.91628  ,
         -75.19927  , -176.85858  ]], dtype=float32), array([[-11.885887  ,   5.761706  ,  49.98755   , ..., -13.98451   ,
          8.55999   ,   6.567243  ],
       [-42.16566   ,   9.647726  ,  39.39186   , ..., -55.859314  ,
         67.70456   ,  47.478096  ],
       [ -1.7547773 ,   0.11256699,   4.8556733 , ..., -17.137312  ,
         33.539433  ,  14.123151  ],
       ...,
    

C:\Users\ethan\AppData\Local\Temp\ipykernel_24316\966333629.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [190]:
net_out = my_net(Variable(test_data))
test_out = torch.max(net_out.data, 1)[1].numpy()

print(np.count_nonzero(test_out==test_labels) / len(test_labels))

0.9528


C:\Users\ethan\AppData\Local\Temp\ipykernel_24316\966333629.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [191]:
print(my_net.fc1.weight.data.numpy().shape)

(50, 784)
